# Tracks dataset cleaning

In [1]:
import numpy as np
import pandas as pd

# sql alchemy for connecting to database
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import inspect
import sqlite3


import os

In [3]:
# read in tracks.csv
df_tracks = pd.read_csv('./tracks.csv')
print(df_tracks.shape)
df_tracks.head(10)

(586672, 20)


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
5,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,0,178933,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.227,0.2610,5,-12.343,1,0.0382,0.994,0.247000,0.0977,0.0539,118.891,4
6,0Dd9ImXtAtGwsmsAD69KZT,La Butte Rouge,0,134467,0,['Francis Marty'],['2nuMRGzeJ5jJEKlfS7rZ0W'],1922,0.510,0.3550,4,-12.833,1,0.1240,0.965,0.000000,0.1550,0.7270,85.754,5
7,0IA0Hju8CAgYfV1hwhidBH,La Java,0,161427,0,['Mistinguett'],['4AxgXfD7ISvJSTObqm4aIE'],1922,0.563,0.1840,4,-13.757,1,0.0512,0.993,0.000016,0.3250,0.6540,133.088,3
8,0IgI1UCz84pYeVetnl1lGP,Old Fashioned Girl,0,310073,0,['Greg Fieler'],['5nWlsH5RDgFuRAiDeOFVmf'],1922,0.488,0.4750,0,-16.222,0,0.0399,0.620,0.006450,0.1070,0.5440,139.952,4
9,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-29,0.548,0.0391,6,-23.228,1,0.1530,0.996,0.933000,0.1480,0.6120,75.595,3


In [4]:
# strip extra brackets and single quotes from artists column
df_tracks['artists'] = df_tracks['artists'].str.strip("['']")

In [5]:
# strip extra brackets and single quotes from id_artists column
df_tracks['id_artists'] = df_tracks['id_artists'].str.strip("['']")

In [6]:
# get first artists only and replace in df
featured_index = df_tracks[df_tracks['artists'].str.contains(',')].index
multiple_artists = df_tracks[df_tracks['artists'].str.contains(',')]
first_artists = multiple_artists['artists'].str.extract(r'((\w*\s)*\w*)\'\,')[0]
df_tracks.loc[df_tracks['artists'].str.contains(','),'artists'] = first_artists

In [7]:
# do the same for artists id 
multiple_id = df_tracks[df_tracks['id_artists'].str.contains(',')]
first_id = df_tracks['id_artists'].str.extract(r'(\w*)\'\,')[0]
df_tracks.loc[df_tracks['id_artists'].str.contains(','),'id_artists'] = first_id

In [8]:
df_tracks.dropna(inplace=True)
df_tracks['release_date'] = df_tracks['release_date'].str.extract(r'(\d{4})')[0]

In [9]:
df_tracks['release_date'] = df_tracks['release_date'].astype(int)

In [10]:
# bin the release_date by decade and place in a list. Append list to dataframe
bin = [1899, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020,2030]
group_decade = ["1900's", "1910's","1920's","1930's","1940's","1950's","1960's","1970's","1980's","1990's","2000's","2010's","2020's"]

df_tracks['decades'] = pd.cut(df_tracks['release_date'], bin, labels=group_decade)

In [11]:
# drop time_signature ==0 or 1
df_tracks.drop(df_tracks[(df_tracks['time_signature'] == 0) | (df_tracks['time_signature'] == 1)].index, inplace=True)

In [12]:
# reset index and print cleaned dataframe
df_tracks.reset_index(drop=True, inplace=True)
df_tracks.head(10)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,decades
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,Uli,45tIt06XoI0Iio4LBEVpls,1922,0.645,0.4450,...,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3,1920's
1,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.434,0.1770,...,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5,1920's
2,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.321,0.0946,...,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3,1920's
3,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.402,0.1580,...,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4,1920's
4,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,0,178933,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.227,0.2610,...,-12.343,1,0.0382,0.994,0.247000,0.0977,0.0539,118.891,4,1920's
5,0Dd9ImXtAtGwsmsAD69KZT,La Butte Rouge,0,134467,0,Francis Marty,2nuMRGzeJ5jJEKlfS7rZ0W,1922,0.510,0.3550,...,-12.833,1,0.1240,0.965,0.000000,0.1550,0.7270,85.754,5,1920's
6,0IA0Hju8CAgYfV1hwhidBH,La Java,0,161427,0,Mistinguett,4AxgXfD7ISvJSTObqm4aIE,1922,0.563,0.1840,...,-13.757,1,0.0512,0.993,0.000016,0.3250,0.6540,133.088,3,1920's
7,0IgI1UCz84pYeVetnl1lGP,Old Fashioned Girl,0,310073,0,Greg Fieler,5nWlsH5RDgFuRAiDeOFVmf,1922,0.488,0.4750,...,-16.222,0,0.0399,0.620,0.006450,0.1070,0.5440,139.952,4,1920's
8,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.548,0.0391,...,-23.228,1,0.1530,0.996,0.933000,0.1480,0.6120,75.595,3,1920's
9,0OYGe21oScKJfanLyM7daU,Capítulo 2.8 - Banquero Anarquista,0,99100,0,Fernando Pessoa,14jtPCOoNZwquk5wd9DxrY,1922,0.676,0.2350,...,-22.447,0,0.9600,0.794,0.000000,0.2100,0.7240,96.777,3,1920's


## Artists dataset cleaning

In [13]:
# read in artists.csv 
file_path = './artists.csv'
df_artists = pd.read_csv(file_path)
df_artists

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0
...,...,...,...,...,...
1104344,6rJIG42vcWAf1UBdRFlQxB,3345.0,[],Cody Longo,8
1104345,1ljurfXKPlGncNdW3J8zJ8,2123.0,['deep acoustic pop'],Right the Stars,18
1104346,2vnT9YhKIvjVo9LnVjWmr2,26.0,[],Jesse Giddings,0
1104347,3ID0E5XCvnJIYZEq043ZoB,406.0,[],The Boy Band Project,0


In [14]:
# remove all empty genres
df_artists = df_artists[df_artists['genres'] != '[]']
df_artists.head()

,id,followers,genres,name,popularity
45,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,['carnaval cadiz'],Las Viudas De Los Bisabuelos,6
46,0dt23bs4w8zx154C5xdVyl,63.0,['carnaval cadiz'],Los De Capuchinos,5
47,0pGhoB99qpEJEsBQxgaskQ,64.0,['carnaval cadiz'],Los “Pofesionales”,7
48,3HDrX2OtSuXLW5dLR85uN3,53.0,['carnaval cadiz'],Los Que No Paran De Rajar,6
136,22mLrN5fkppmuUPsHx6i2G,59.0,"['classical harp', 'harp']",Vera Dulova,3


In [15]:
# convert genres column into a list datatype
df_artists['genres'] = df_artists['genres'].apply(eval)
df_artists.head()

<ipython-input-15-d7974bac8c18>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_artists['genres'] = df_artists['genres'].apply(eval)


,id,followers,genres,name,popularity
45,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,[carnaval cadiz],Las Viudas De Los Bisabuelos,6
46,0dt23bs4w8zx154C5xdVyl,63.0,[carnaval cadiz],Los De Capuchinos,5
47,0pGhoB99qpEJEsBQxgaskQ,64.0,[carnaval cadiz],Los “Pofesionales”,7
48,3HDrX2OtSuXLW5dLR85uN3,53.0,[carnaval cadiz],Los Que No Paran De Rajar,6
136,22mLrN5fkppmuUPsHx6i2G,59.0,"[classical harp, harp]",Vera Dulova,3


In [16]:
# reset index and print cleaned dataframe
df_artists.reset_index(drop=True, inplace=True)
df_artists.head()

,id,followers,genres,name,popularity
0,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,[carnaval cadiz],Las Viudas De Los Bisabuelos,6
1,0dt23bs4w8zx154C5xdVyl,63.0,[carnaval cadiz],Los De Capuchinos,5
2,0pGhoB99qpEJEsBQxgaskQ,64.0,[carnaval cadiz],Los “Pofesionales”,7
3,3HDrX2OtSuXLW5dLR85uN3,53.0,[carnaval cadiz],Los Que No Paran De Rajar,6
4,22mLrN5fkppmuUPsHx6i2G,59.0,"[classical harp, harp]",Vera Dulova,3


In [17]:
# check that conversion worked
print(type(df_artists.loc[0,'genres']))

<class 'list'>


In [18]:
# define function to create a series of value counts for unique genres
def unique_values(series):
    return pd.Series([x for _list in series for x in _list])

In [19]:
# apply function to genres column in artists dataframe
unique = unique_values(df_artists['genres']).value_counts()
unique

dance pop                572
pop                      568
rock                     564
electro house            558
classical performance    502
                        ... 
musica puntana             3
himene tarava              2
mazandarani folk           2
musica timor-leste         2
swazi traditional          1
Length: 5365, dtype: int64

In [20]:
# create values columns
unique = pd.Series(unique, name="values")
unique

dance pop                572
pop                      568
rock                     564
electro house            558
classical performance    502
                        ... 
musica puntana             3
himene tarava              2
mazandarani folk           2
musica timor-leste         2
swazi traditional          1
Name: values, Length: 5365, dtype: int64

In [21]:
# create unique genres dataframe
unique_df = pd.DataFrame(unique)
unique_df.head()

,values
dance pop,572
pop,568
rock,564
electro house,558
classical performance,502


In [22]:
# format unique dataframe
unique_df.reset_index(inplace=True)
unique_df.rename(columns={'index': 'genres'}, inplace=True)
unique_df.head()

,genres,values
0,dance pop,572
1,pop,568
2,rock,564
3,electro house,558
4,classical performance,502


In [23]:
# explode genres column in artists dataframe to get individual row per song genre
df_artists = df_artists.explode('genres')
df_artists.head()

,id,followers,genres,name,popularity
0,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,carnaval cadiz,Las Viudas De Los Bisabuelos,6
1,0dt23bs4w8zx154C5xdVyl,63.0,carnaval cadiz,Los De Capuchinos,5
2,0pGhoB99qpEJEsBQxgaskQ,64.0,carnaval cadiz,Los “Pofesionales”,7
3,3HDrX2OtSuXLW5dLR85uN3,53.0,carnaval cadiz,Los Que No Paran De Rajar,6
4,22mLrN5fkppmuUPsHx6i2G,59.0,classical harp,Vera Dulova,3


In [24]:
# merge artists and unqiue genres dataframes to get value counts
df_artists = df_artists.merge(unique_df, how='left', on='genres')
df_artists.head()

,id,followers,genres,name,popularity,values
0,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,carnaval cadiz,Las Viudas De Los Bisabuelos,6,69
1,0dt23bs4w8zx154C5xdVyl,63.0,carnaval cadiz,Los De Capuchinos,5,69
2,0pGhoB99qpEJEsBQxgaskQ,64.0,carnaval cadiz,Los “Pofesionales”,7,69
3,3HDrX2OtSuXLW5dLR85uN3,53.0,carnaval cadiz,Los Que No Paran De Rajar,6,69
4,22mLrN5fkppmuUPsHx6i2G,59.0,classical harp,Vera Dulova,3,82


In [25]:
# sort values column in ascending order
df_artists.sort_values(by=['values'], inplace=True)
df_artists.head()

,id,followers,genres,name,popularity,values
185854,05YcBmsjblLBb6oROMrwqF,78.0,swazi traditional,Ipelegeng Ensemble,19,1
382469,72PYDG3xLIxeRagDWDcss6,29.0,himene tarava,Tubuai Choir,0,2
360254,43gNFp9nIKgWbu8DNz24Nc,8.0,himene tarava,Himene Tarava,0,2
324978,4WsBfMeAqIoAiHK7n80ydX,146.0,mazandarani folk,Fereydoon Pour Reza,1,2
322797,6ZU1htIEHw1tA3VzHcgiJl,90.0,musica timor-leste,The Dili Allstars,3,2


In [26]:
# drop duplicates in id column and only keep last occurance (highest value)
df_artists.drop_duplicates(['id'], keep='last', inplace=True)
df_artists.head()

,id,followers,genres,name,popularity,values
360254,43gNFp9nIKgWbu8DNz24Nc,8.0,himene tarava,Himene Tarava,0,2
324978,4WsBfMeAqIoAiHK7n80ydX,146.0,mazandarani folk,Fereydoon Pour Reza,1,2
322797,6ZU1htIEHw1tA3VzHcgiJl,90.0,musica timor-leste,The Dili Allstars,3,2
272075,0MWnEq4x3G8ktrdN1tPEeX,141.0,musica timor-leste,Ego Lemos,2,2
239259,2OOTxuWRlzN2BQMvOfpolB,32.0,mazandarani folk,"Abolhasan Khoshru (Avaz & Desarkotan), Abdolla...",0,2


In [27]:
# drop to values column in the artists dataframe
df_artists.drop(columns=['values'], inplace=True)
df_artists.head()

,id,followers,genres,name,popularity
360254,43gNFp9nIKgWbu8DNz24Nc,8.0,himene tarava,Himene Tarava,0
324978,4WsBfMeAqIoAiHK7n80ydX,146.0,mazandarani folk,Fereydoon Pour Reza,1
322797,6ZU1htIEHw1tA3VzHcgiJl,90.0,musica timor-leste,The Dili Allstars,3
272075,0MWnEq4x3G8ktrdN1tPEeX,141.0,musica timor-leste,Ego Lemos,2
239259,2OOTxuWRlzN2BQMvOfpolB,32.0,mazandarani folk,"Abolhasan Khoshru (Avaz & Desarkotan), Abdolla...",0


In [28]:
# reset the index 
df_artists.reset_index(drop=True, inplace=True)
df_artists.head()

,id,followers,genres,name,popularity
0,43gNFp9nIKgWbu8DNz24Nc,8.0,himene tarava,Himene Tarava,0
1,4WsBfMeAqIoAiHK7n80ydX,146.0,mazandarani folk,Fereydoon Pour Reza,1
2,6ZU1htIEHw1tA3VzHcgiJl,90.0,musica timor-leste,The Dili Allstars,3
3,0MWnEq4x3G8ktrdN1tPEeX,141.0,musica timor-leste,Ego Lemos,2
4,2OOTxuWRlzN2BQMvOfpolB,32.0,mazandarani folk,"Abolhasan Khoshru (Avaz & Desarkotan), Abdolla...",0


In [29]:
# create engine and connect to the spotify database
engine = create_engine("sqlite:///spotify_db.db")

# map the tables to objects
orm = automap_base()
orm.prepare(engine, reflect=True)
inspector = inspect(engine)


In [30]:
# check for existing tables in the database
inspector.get_table_names()

[]

In [31]:
# create artists table in sqlite database
engine.execute('''
CREATE TABLE artists (
     id VARCHAR(22) NOT NULL,
     followers INT,
	 genres VARCHAR(317),
	 name VARCHAR(194) NOT NULL,
	 popularity INT,
     PRIMARY KEY (id))
     ''')

In [32]:
# create tracks table in sqlite database
engine.execute('''
CREATE TABLE tracks (
	 id VARCHAR(22),
	 name VARCHAR(529),
	 popularity INT,
	 duration_ms INT,
	 explicit INT,
	 artists VARCHAR(100), 
	 id_artists VARCHAR(22),
	 release_date VARCHAR(4),
	 danceability REAL,
	 energy REAL,
	 key INT,
	 loudness REAL,
	 mode INT,
	 speechiness REAL,
	 acousticness REAL,
	 instrumentalness REAL,
	 liveness REAL,
	 valence REAL,
	 tempo REAL,
	 time_signature INT,
	 decades VARCHAR(6),
     PRIMARY KEY (id))
     ''')

In [33]:
# check for artists columns and table schema
columns = inspector.get_columns('artists')
for column in columns:
    print(column)

{'name': 'id', 'type': VARCHAR(length=22), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'followers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'genres', 'type': VARCHAR(length=317), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'name', 'type': VARCHAR(length=194), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'popularity', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}


In [34]:
# check for tracks columns and table schema
columns = inspector.get_columns('tracks')
for column in columns:
    print(column)

{'name': 'id', 'type': VARCHAR(length=22), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'name', 'type': VARCHAR(length=529), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'popularity', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'duration_ms', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'explicit', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'artists', 'type': VARCHAR(length=100), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'id_artists', 'type': VARCHAR(length=22), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'release_date', 'type': VARCHAR(length=4), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': '

In [35]:
# write df_tracks to tracks table in spotify database
df_tracks.to_sql('tracks', engine, if_exists='append', index = False, chunksize=100)

In [36]:
# write df_artists to artists table in spotify database
df_artists.to_sql('artists', engine, if_exists='append', index = False, chunksize=100)

In [37]:
conn = sqlite3.connect("spotify_db.db")
cursor = conn.cursor()

In [38]:
cursor.execute('''
CREATE TABLE master (
	 id VARCHAR(22),
	 name VARCHAR(529),
	 popularity INT,
	 duration_ms INT,
	 explicit INT,
	 artists VARCHAR(100), 
	 id_artists VARCHAR(22),
	 release_date VARCHAR(4),
	 danceability REAL,
	 energy REAL,
	 key INT,
	 loudness REAL,
	 mode INT,
	 speechiness REAL,
	 acousticness REAL,
	 instrumentalness REAL,
	 liveness REAL,
	 valence REAL,
	 tempo REAL,
	 time_signature INT,
	 decades VARCHAR(6),
	 followers INT,
	 genres VARCHAR(317),
	 PRIMARY KEY (id, id_artists))
     ''')

In [39]:
cursor.execute("""INSERT INTO master SELECT A.id, A.name, A.popularity, A.duration_ms,A.explicit, A.artists, A.id_artists, 
           A.release_date, A.danceability, A.energy, A.key, A.loudness, A.mode, A.speechiness, A.acousticness, 
           A.instrumentalness, A.liveness, A.valence, A.tempo, A.time_signature, A.decades, B.followers, B.genres 
           FROM tracks A
           INNER JOIN artists B 
           ON A.id_artists=B.id""")       

conn.commit()

In [40]:
inspector.get_table_names()

[]

In [41]:
# check for master columns and table schema
columns = inspector.get_columns('master')
for column in columns:
    print(column)

{'name': 'id', 'type': VARCHAR(length=22), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'name', 'type': VARCHAR(length=529), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'popularity', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'duration_ms', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'explicit', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'artists', 'type': VARCHAR(length=100), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'id_artists', 'type': VARCHAR(length=22), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 2}
{'name': 'release_date', 'type': VARCHAR(length=4), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': '

In [42]:
master_df = pd.read_sql_query("SELECT * FROM master", conn)

In [43]:
master_df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,decades,followers,genres
0,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.434,0.1770,...,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5,1920's,3528.0,tango
1,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.321,0.0946,...,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3,1920's,3528.0,tango
2,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.402,0.1580,...,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4,1920's,11327.0,adult standards
3,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,0,178933,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.227,0.2610,...,0.0382,0.994,0.247000,0.0977,0.0539,118.891,4,1920's,11327.0,adult standards
4,0IA0Hju8CAgYfV1hwhidBH,La Java,0,161427,0,Mistinguett,4AxgXfD7ISvJSTObqm4aIE,1922,0.563,0.1840,...,0.0512,0.993,0.000016,0.3250,0.6540,133.088,3,1920's,5078.0,vintage chanson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518666,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,阿YueYue,1QLBXKM5GCpyQQSVMNZqrZ,2020,0.560,0.5180,...,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4,2010's,896.0,chinese viral pop
518667,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,ROLE MODEL,1dy5WNgIKQU6ezkpZs4y8z,2020,0.765,0.6630,...,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4,2010's,245944.0,pop
518668,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,FINNEAS,37M5pPGs6V1fchFJSgCguX,2020,0.535,0.3140,...,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4,2010's,1168213.0,pop
518669,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,Gentle Bones,4jGPdu95icCKVF31CcFKbS,2021,0.696,0.6150,...,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4,2020's,45309.0,indie cafe pop
